In [37]:
# generator
import keras
from keras import layers
import numpy as np
from keras.utils import plot_model 

latent_dim = 32
height = 32
width = 32
channels = 3

In [38]:
generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)

plot_model(generator, to_file='generator_plot.png', show_shapes=True)

![generator](generator_plot.png)

In [39]:
# discriminator
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)
x = layers.Dropout(0.6)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)

plot_model(discriminator, to_file='discriminator_plot.png', show_shapes=True)

![discriminator](discriminator_plot.png)

In [40]:
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [41]:
# adversarial network
discriminator_trainable = False

gan_input = generator_input
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

plot_model(gan, 'gan_plot.png', show_shapes=True)

![gan](gan_plot.png)

In [42]:
# training
import os
from keras.preprocessing import image

(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 6] # select frog images
x_train = x_train.reshape((x_train.shape[0], ) + (height, width, channels)).astype('float32') / 255

iterations = 10000
batch_size = 20
save_dir = 'generated_imgs'

In [43]:
start = 0
for step in range(iterations):
    # for training discriminator
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    generated_imgs = generator.predict_on_batch(random_latent_vectors)
    
    stop = start + batch_size
    
    real_imgs = x_train[start: stop]
    combined_imgs = np.concatenate([generated_imgs, real_imgs])
    
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    
    d_loss = discriminator.train_on_batch(combined_imgs, labels)
    
    # for training generator
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    
    if start > len(x_train) - batch_size:
        start = 0
    
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        print('step: ', step)

        print('discriminator loss: ', d_loss)
        print('adversarial loss: ', a_loss)

        img = image.array_to_img(generated_imgs[-1] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_img' + str(step) + '.png'))

        img = image.array_to_img(real_imgs[-1] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_img' + str(step) + '.png'))

step:  0
discriminator loss:  0.696667
adversarial loss:  0.647453


KeyboardInterrupt: 